In [1]:
import keras
import random
import tensorflow as tf
import numpy as np
from tensorflow.keras import layers            
from tensorflow.keras.layers import AveragePooling2D,Dense,Dropout
from tensorflow.keras.models import Model
from tensorflow.keras import regularizers
from tensorflow.keras import backend as K
from sklearn.model_selection import KFold
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras import layers,optimizers,losses
from sklearn.metrics import accuracy_score,precision_score,recall_score,f1_score
from keras.layers import Activation
from tensorflow.keras.layers import Input,Flatten,LSTM
from tensorflow.keras.layers import Embedding, Layer 

In [2]:
# 设置GPU使用方式
# 获取GPU列表
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        # 设置GPU为增长式占用
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu,True)
    except RuntimeError as e:
        #打印异常
        print(e)

In [3]:
batchsz = 16
#加载数据
#1）一次性读入全部数据到内存
data_all=np.load('D:/An/脑电/实验数据/epoch/训练前后分类/xunlian.npy')
#data_alln=np.expand_dims(x_train,axis=3)
label_all=np.concatenate((np.zeros((4797,1),dtype=int),np.ones((4802,1),dtype=int)),axis=0)
label_all=np.eye(2)[label_all]
label_all=np.squeeze(label_all)
n=len(data_all)
A = np.linspace(0,n-1,n,dtype=int)
random.shuffle(A)
data_all=data_all[A]
label_all=label_all[A]
#print(data_all,label_all.shape)
#print(n)
kfold=KFold(n_splits=10, shuffle=True, random_state=1)
# 创建所有要保存结果的空列表
historys,test_pred,test_real,accuracy,precision,recall,f1score=list(),list(),list(),list(),list(),list(),list()


In [4]:
class TimeEmbeddingLayer(Layer):
    def __init__(self, num_channels, sequence_length, output_dim, **kwargs):
        super(TimeEmbeddingLayer, self).__init__(**kwargs)
        self.num_channels = num_channels
        self.sequence_length = sequence_length
        self.output_dim = output_dim
        self.time_embedding_layer = Embedding(input_dim=sequence_length, output_dim=output_dim)

    def call(self, inputs):
        position = tf.range(self.sequence_length)
        embed = self.time_embedding_layer(position)
        embed = tf.reshape(embed, (1, 1, self.sequence_length, self.output_dim))
        return inputs + embed
    
    def get_config(self):
        config = super(TimeEmbeddingLayer, self).get_config()
        config.update({
            "num_channels": self.num_channels,
            "sequence_length": self.sequence_length,
            "output_dim": self.output_dim
        })
        return config

class ChannelEmbeddingLayer(Layer):
    def __init__(self, num_channels, sequence_length, output_dim, **kwargs):
        super(ChannelEmbeddingLayer, self).__init__(**kwargs)
        self.num_channels = num_channels
        self.sequence_length = sequence_length
        self.output_dim = output_dim
        self.channel_embedding_layer = Embedding(input_dim=num_channels, output_dim=output_dim)
        
    def call(self, inputs):
        position = tf.range(self.num_channels)
        embed = self.channel_embedding_layer(position)
        embed = tf.reshape(embed, (1, self.num_channels, 1, self.output_dim))
        return inputs + embed
    
    def get_config(self):
        config = super(ChannelEmbeddingLayer, self).get_config()
        config.update({
            "num_channels": self.num_channels,
            "sequence_length": self.sequence_length,
            "output_dim": self.output_dim
        })
        return config

def mymodel1():
    input = layers.Input(shape=(30,36,40))
    x=TimeEmbeddingLayer(num_channels=30, sequence_length=36, output_dim=40)(input)
    x=layers.Conv2D(40,kernel_size=(30,1),strides=(1,1))(x)
    x=layers.BatchNormalization()(x)
    x=layers.ReLU()(x)
    x=layers.Reshape((x.shape[2],x.shape[3]))(x)
    attn_input=x
    x=layers.MultiHeadAttention(num_heads=8,key_dim=40)(x,x)
    x=layers.Add()([x,attn_input])
    x=layers.LSTM(units=100, return_sequences=True)(x)
    model = Model(inputs=input, outputs=x)
    return model

def mymodel2():
    input = layers.Input(shape=(30,36,40))
    x=ChannelEmbeddingLayer(num_channels=30, sequence_length=36, output_dim=40)(input)
    x=layers.Conv2D(40, kernel_size=(1,36), strides=(1, 1))(x)
    x=layers.BatchNormalization()(x)
    x=layers.ReLU()(x)
    x=layers.Reshape((x.shape[1],x.shape[3]))(x)
    attn_input=x
    x=layers.MultiHeadAttention(num_heads=8,key_dim=40)(x,x)
    x=layers.Add()([x,attn_input])
    x=layers.LSTM(units=100, return_sequences=True)(x)
    model = Model(inputs=input, outputs=x)
    return model

def siamese_network(inp_shape=(30,200,1)):
    inp=Input(shape=inp_shape)
    x=layers.Conv2D(40,kernel_size=(1,15),strides=(1,1),
                      name='conv_1d_temporal',
                      kernel_regularizer=regularizers.l2(0.01),
                      bias_regularizer=regularizers.l2(0.01)
                      )(inp)
    x=layers.BatchNormalization()(x)
    x=layers.ReLU()(x)
    x=AveragePooling2D(pool_size=(1,10),strides=(1,5))(x)
    out1=mymodel1()(x)
    out2=mymodel2()(x)
    merged=tf.keras.layers.concatenate([out1,out2],axis=1)
    x=layers.Flatten()(merged)
    x=layers.Dense(100, activation="relu")(x)
    x=layers.Dropout(0.5)(x)
    x=layers.Dense(50, activation="relu")(x)
    model = Model(inputs=inp, outputs=x)
    return model

def mymodel(nb_classes=2, dropoutRate=0.5):
    inp_shape=(2,30,200,1)
    inp = layers.Input(shape=inp_shape)
    split1,split2=tf.unstack(inp,axis=1)
    siamese=siamese_network(inp_shape=(30, 200, 1))
    feature1=siamese(split1)
    feature2=siamese(split2)
    x=layers.Add()([feature1, feature2])
    x=layers.Dense(100, activation='relu')(x)
    x=layers.Dropout(dropoutRate)(x)
    x=layers.Dense(50, activation='relu')(x)
    output=layers.Dense(nb_classes, activation='softmax')(x)
    model=keras.models.Model(inputs=inp,outputs=output)
    return model

In [5]:
# 创建十折交叉验证中显示第几折的指示变量
ind_fold=0
# 进行十折交叉验证
for train_ind,test_ind in kfold.split(data_all,label_all):
    # 显示训练到第几折
    ind_fold=ind_fold+1
    print('fold hao:',ind_fold)
    # 每一折验证前都要打乱训练集样本顺序
    n=len(train_ind)
    A=np.linspace(0,n-1,n,dtype=int)
    random.shuffle(A)
    # 构建训练集、验证集、测试集
    epoch_train=data_all[train_ind[A[:int(0.8*n)]]]
    epoch_val=data_all[train_ind[A[int(0.8*n):]]]
    epoch_test=data_all[test_ind]
    label_train=label_all[train_ind[A[:int(0.8*n)]]]
    label_val=label_all[train_ind[A[int(0.8*n):]]]
    label_test=label_all[test_ind]

    print(epoch_train.shape)
    print(label_train.shape)
    
    db_train=tf.data.Dataset.from_tensor_slices((epoch_train,label_train))
    db_val=tf.data.Dataset.from_tensor_slices((epoch_val,label_val))
    db_train=db_train.shuffle(1000).batch(batchsz)
    db_val=db_val.shuffle(1000).batch(batchsz)

    # 选择、创建模型
    model=mymodel(nb_classes=2,dropoutRate=0.3)  
    # 显示模型结构
    model.summary()
    # 创建Early Stopping类，连续3次不下降则终止
    early_stopping = EarlyStopping(monitor='val_loss',min_delta=0.001,patience=10)
    reduce_lr=keras.callbacks.ReduceLROnPlateau(monitor="val_loss",patience=5)
    # 配置模型训练
    model.compile(optimizer=optimizers.Adam(lr=1e-4),
                   loss=losses.MSE,
                   metrics=['accuracy'])
    # 开始训练模型
    print('开始训练!!') 
    history=model.fit(db_train, validation_data=db_val, validation_freq=1,  
                         shuffle=True, epochs=100,callbacks=[early_stopping,reduce_lr])
    # 提取和保存训练记录
    history = history.history
    historys.append(history)
    # # 计算、保存测试结果
    pred_test=model.predict(epoch_test)
    pred_test=np.rint(pred_test)
    print(pred_test.shape)
    print(label_test.shape)
    # 保存测试集预测结果和真实结果
    test_pred.append(pred_test)
    test_real.append(label_test)
    # 计算准确率，精确率，召回率，f1评分
    acc=accuracy_score(pred_test,label_test)
    pre=precision_score(pred_test,label_test,average='macro')
    rec=recall_score(pred_test,label_test,average='macro')
    f1=f1_score(pred_test,label_test,average='macro')
    accuracy.append(acc)
    precision.append(pre)
    recall.append(rec)
    f1score.append(f1)
    print(f"$$ 测试集准确率为 accuracy:{acc}")
    print(f"$$ 测试集精确率为 precision:{pre}")
    print(f"$$ 测试集召回率为 recall:{rec}")
    print(f"$$ 测试集f1评分为 f1_score:{f1}")
# 将每一折history中误差结果保存（训练集和测试集，用于反映训练过程）    
loss_train=[]
loss_val=[]
for history_s in historys:
    loss_val.append(history_s['val_loss'])
    loss_train.append(history_s['loss'])

print(f"$$ 测试集准确率为 accuracy:{np.mean(accuracy)}")
print(f"$$ 测试集精确率为 precision:{np.mean(precision)}")
print(f"$$ 测试集召回率为 recall:{np.mean(recall)}")
print(f"$$ 测试集f1评分为 f1_score:{np.mean(f1score)}")

fold hao: 1
(6911, 2, 30, 200)
(6911, 2)
Model: "model_3"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 2, 30, 200,  0           []                               
                                 1)]                                                              
                                                                                                  
 tf.unstack (TFOpLambda)        [(None, 30, 200, 1)  0           ['input_1[0][0]']                
                                , (None, 30, 200, 1                                               
                                )]                                                                
                                                                                                  
 model_2 (Functional)           (None, 50)         

d:\Anaconda3\envs\tensorflow\lib\site-packages\keras\optimizer_v2\adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


Epoch 1/100
432/432 [==============================] - 26s 32ms/step - loss: 0.2125 - accuracy: 0.6895 - val_loss: 0.1112 - val_accuracy: 0.8698 - lr: 1.0000e-04
Epoch 2/100
432/432 [==============================] - 11s 25ms/step - loss: 0.1095 - accuracy: 0.8780 - val_loss: 0.1588 - val_accuracy: 0.8021 - lr: 1.0000e-04
Epoch 3/100
432/432 [==============================] - 11s 25ms/step - loss: 0.0769 - accuracy: 0.9197 - val_loss: 0.0602 - val_accuracy: 0.9427 - lr: 1.0000e-04
Epoch 4/100
432/432 [==============================] - 11s 25ms/step - loss: 0.0629 - accuracy: 0.9400 - val_loss: 0.0684 - val_accuracy: 0.9265 - lr: 1.0000e-04
Epoch 5/100
432/432 [==============================] - 12s 27ms/step - loss: 0.0541 - accuracy: 0.9499 - val_loss: 0.0501 - val_accuracy: 0.9560 - lr: 1.0000e-04
Epoch 6/100
432/432 [==============================] - 11s 26ms/step - loss: 0.0470 - accuracy: 0.9604 - val_loss: 0.0486 - val_accuracy: 0.9583 - lr: 1.0000e-04
Epoch 7/100
432/432 [=======

d:\Anaconda3\envs\tensorflow\lib\site-packages\keras\optimizer_v2\adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


Epoch 1/100
432/432 [==============================] - 15s 27ms/step - loss: 0.2188 - accuracy: 0.6752 - val_loss: 0.1479 - val_accuracy: 0.8108 - lr: 1.0000e-04
Epoch 2/100
432/432 [==============================] - 11s 25ms/step - loss: 0.1101 - accuracy: 0.8701 - val_loss: 0.0669 - val_accuracy: 0.9311 - lr: 1.0000e-04
Epoch 3/100
432/432 [==============================] - 11s 25ms/step - loss: 0.0818 - accuracy: 0.9114 - val_loss: 0.0582 - val_accuracy: 0.9416 - lr: 1.0000e-04
Epoch 4/100
432/432 [==============================] - 11s 25ms/step - loss: 0.0660 - accuracy: 0.9343 - val_loss: 0.0803 - val_accuracy: 0.9178 - lr: 1.0000e-04
Epoch 5/100
432/432 [==============================] - 11s 25ms/step - loss: 0.0563 - accuracy: 0.9443 - val_loss: 0.0484 - val_accuracy: 0.9549 - lr: 1.0000e-04
Epoch 6/100
432/432 [==============================] - 11s 25ms/step - loss: 0.0491 - accuracy: 0.9547 - val_loss: 0.0590 - val_accuracy: 0.9416 - lr: 1.0000e-04
Epoch 7/100
432/432 [=======

...

Epoch 1/100
432/432 [==============================] - 15s 27ms/step - loss: 0.2174 - accuracy: 0.6694 - val_loss: 0.1266 - val_accuracy: 0.8443 - lr: 1.0000e-04
Epoch 2/100
432/432 [==============================] - 11s 25ms/step - loss: 0.1210 - accuracy: 0.8563 - val_loss: 0.1039 - val_accuracy: 0.8802 - lr: 1.0000e-04
Epoch 3/100
432/432 [==============================] - 11s 26ms/step - loss: 0.0864 - accuracy: 0.9067 - val_loss: 0.1437 - val_accuracy: 0.8356 - lr: 1.0000e-04
Epoch 4/100
432/432 [==============================] - 11s 26ms/step - loss: 0.0695 - accuracy: 0.9314 - val_loss: 0.1158 - val_accuracy: 0.8750 - lr: 1.0000e-04
Epoch 5/100
432/432 [==============================] - 11s 26ms/step - loss: 0.0596 - accuracy: 0.9431 - val_loss: 0.1194 - val_accuracy: 0.8559 - lr: 1.0000e-04
Epoch 6/100
432/432 [==============================] - 11s 26ms/step - loss: 0.0511 - accuracy: 0.9533 - val_loss: 0.0383 - val_accuracy: 0.9676 - lr: 1.0000e-04
Epoch 7/100
432/432 [=======

d:\Anaconda3\envs\tensorflow\lib\site-packages\keras\optimizer_v2\adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


Epoch 1/100
432/432 [==============================] - 16s 28ms/step - loss: 0.2253 - accuracy: 0.6532 - val_loss: 0.1620 - val_accuracy: 0.7870 - lr: 1.0000e-04
Epoch 2/100
432/432 [==============================] - 11s 26ms/step - loss: 0.1157 - accuracy: 0.8659 - val_loss: 0.0635 - val_accuracy: 0.9387 - lr: 1.0000e-04
Epoch 3/100
432/432 [==============================] - 12s 28ms/step - loss: 0.0775 - accuracy: 0.9177 - val_loss: 0.0769 - val_accuracy: 0.9213 - lr: 1.0000e-04
Epoch 4/100
432/432 [==============================] - 11s 27ms/step - loss: 0.0644 - accuracy: 0.9365 - val_loss: 0.0572 - val_accuracy: 0.9450 - lr: 1.0000e-04
Epoch 5/100
432/432 [==============================] - 12s 27ms/step - loss: 0.0555 - accuracy: 0.9491 - val_loss: 0.0590 - val_accuracy: 0.9439 - lr: 1.0000e-04
Epoch 6/100
432/432 [==============================] - 12s 27ms/step - loss: 0.0516 - accuracy: 0.9512 - val_loss: 0.0539 - val_accuracy: 0.9485 - lr: 1.0000e-04
Epoch 7/100
432/432 [=======

d:\Anaconda3\envs\tensorflow\lib\site-packages\keras\optimizer_v2\adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


Epoch 1/100
432/432 [==============================] - 16s 28ms/step - loss: 0.2230 - accuracy: 0.6599 - val_loss: 0.1312 - val_accuracy: 0.8362 - lr: 1.0000e-04
Epoch 2/100
432/432 [==============================] - 11s 26ms/step - loss: 0.1164 - accuracy: 0.8621 - val_loss: 0.0730 - val_accuracy: 0.9230 - lr: 1.0000e-04
Epoch 3/100
432/432 [==============================] - 11s 26ms/step - loss: 0.0803 - accuracy: 0.9129 - val_loss: 0.0539 - val_accuracy: 0.9543 - lr: 1.0000e-04
Epoch 4/100
432/432 [==============================] - 12s 27ms/step - loss: 0.0617 - accuracy: 0.9372 - val_loss: 0.0433 - val_accuracy: 0.9612 - lr: 1.0000e-04
Epoch 5/100
432/432 [==============================] - 13s 30ms/step - loss: 0.0546 - accuracy: 0.9478 - val_loss: 0.0386 - val_accuracy: 0.9676 - lr: 1.0000e-04
Epoch 6/100
432/432 [==============================] - 13s 29ms/step - loss: 0.0452 - accuracy: 0.9609 - val_loss: 0.0337 - val_accuracy: 0.9757 - lr: 1.0000e-04
Epoch 7/100
432/432 [=======